# Finding Independent Coffee

Use Google Place Search and Wikipedia to find coffeeshops that probably aren't chains.

In [1]:
import json
from urllib.request import urlopen

goog_key = ''

def google_place_search(location, radius, keyword):
    coords = str(location[0]) + ',' + str(location[1])
    url_keyword = keyword.replace(' ', '%20')
    url = ('https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
           'location=%s'
           '&radius=%s'
           '&keyword=%s'
           '&key=%s') % (coords, radius, url_keyword, goog_key)
    response = urlopen(url).read().decode('utf-8')
    json_data = json.loads(response)
    return json_data

## Inputs

### Coffee Chain Data

source: https://en.wikipedia.org/wiki/List_of_coffeehouse_chains

"This list excludes the many companies which operate coffeeshops within retail establishments, notably bookstores and department stores, or restaurants or convenience stores which also serve coffee."

In [2]:
chains = set(line.strip() for line in open('coffee_chains.txt'))
chains

{'85C Bakery Cafe',
 'A Twosome Place',
 'AMT Coffee',
 'Aida',
 'Angel-in-us',
 'Arcaffe',
 'Aroma Café',
 'Aroma Espresso Bar',
 'Au Bon Pain',
 "Baker's Dozen Donuts",
 'Barista Lavazza',
 'Baristas',
 'Bess Eaton',
 "Bewley's",
 'Biggby Coffee',
 'Blenz Coffee',
 'Blue Bottle Coffee Company',
 'Blue State Coffee',
 'Bourbon Coffee',
 'Bridgehead Coffee',
 'Cadena Cafes Limited',
 'Cafe Barbera',
 'Caffe Bene',
 'Caffe Luxxe',
 'Caffe Trieste',
 'Caffè Cova',
 'Caffè Nero',
 'Caffè Pascucci',
 'Caffè Ritazza',
 'Caffè Umbria',
 'Caffé Vita Coffee Roasting Company',
 'Café A Brasileira',
 'Café Café',
 'Café Coffee Day',
 'Café Hillel',
 'Café Picnic',
 'Café du Monde',
 'Caribou Coffee',
 'Cibo Espresso',
 'Cinnzeo',
 'Coffee Beanery',
 'Coffee Island',
 'Coffee Republic',
 'Coffee Time',
 'Coffee World',
 'Coffee#1',
 'Coffeeshop Company',
 'Coffine Gurunaru',
 'Cofix',
 'Colectivo Coffee Roasters',
 'Costa Coffee',
 'Country Style',
 'Crazy Mocha Coffee Company',
 "Cuppy's Coffee"

### Location Variables

In [3]:
search_location = [41.7627, -72.6743] # Hartford, CT
search_radius = 25000 # meters

## Query

In [4]:
results = google_place_search(search_location,search_radius,'coffee')
coffeeshops = results['results']

In [5]:
for location in coffeeshops:
    if location['name'] in chains:
        location['independent'] = False
    else:
        location['independent'] = True
coffeeshops

[{'geometry': {'location': {'lat': 41.7668756, 'lng': -72.675966}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png',
  'id': '0bf62176c95165c96dab1e6842037749de1ff3f6',
  'independent': False,
  'name': 'Starbucks',
  'opening_hours': {'open_now': True, 'weekday_text': []},
  'photos': [{'height': 3120,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105378355845589571550/photos">Kurt Loescher</a>'],
    'photo_reference': 'CmRdAAAA8hgwfmv6kUfsSDq4SsnPLHKl4blgda6q2z1QNJ7_Fhnrtpys3pv9kQnNkzEhwKuRgdlVmTxVD79a1ZpTDdHf8CpjXjw78XSbAFPv0XqlBrdQhCyMfGc3v8tK9Hmlo0e4EhAen49wkc4Zwjih3VDw6Kl9GhSWBXQvL8X8P-UdG1yUGpfWE3Dsvg',
    'width': 4160}],
  'place_id': 'ChIJbW6s8mRT5okRR-GJkSaej1c',
  'price_level': 2,
  'rating': 4.3,
  'reference': 'CmRcAAAAt-vEJ51nO9Ni_8X0kPzPpGkFfJT0u_91e5FIaJDpNApVEOQDBtDvnWjq3C0emoiEltiujN6SOEzKCg0aab0GKkVG8vHAWMvbAghXdJCqcUqrb6chkynou_Jsdka7YjJAEhDiPt5t8CEE_JL8KTZZBD-MGhTitXFwKYpP33rtA9u2o7Iao-GUZA',
  'scope': 'GOOG

In [6]:
independents = []
chains = []

for location in coffeeshops:
    if location['independent']:
        independents.append(location)
    else:
        chains.append(location)

## Results

In [7]:
num_independents = len(independents)
num_chains = len(chains)
percent_indep = (num_independents / len(coffeeshops)) * 100

print(str(num_independents) + " independent shops")
print(str(num_chains) + " chain shops")
print(str(percent_indep) + "% independent shops")

17 independent shops
3 chain shops
85.0% independent shops


### Totals by chain

In [8]:
chain_names = []
for shop in chains:
    chain_names.append(shop['name'])

brands = set(chain_names) # reduce chain shops to unique brands
chain_totals = {name:0 for name in brands}

for shop in chain_names:
    chain_totals[shop] = chain_totals[shop] + 1
chain_totals

{'Blue State Coffee': 1, "Dunkin' Donuts": 1, 'Starbucks': 1}

### Map

In [32]:
import folium

map = folium.Map(location = search_location,
                zoom_start = 10)

In [34]:
for entry in independents:
    coords = entry['geometry']['location']
    map.simple_marker(
        location = [coords['lat'],
                   coords['lng']],
        popup = entry['name'])

map

/home/jupyter/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: simple_marker is deprecated. Use add_children(Marker) instead
